In [ ]:
!pip uninstall -y eonacs && pip install git+https://github.com/njnmco/smc #--log /dev/stderr

In [2]:
import pandas as pd
import numpy as np

In [3]:
import eonacs.common.util as util

util.colab_map_drive()


Mounted at /content/drive


In [4]:
model_path = "/content/drive/MyDrive/smc/models/dbert_tasks_pac_refactor"

In [5]:
import eonacs.common.dbert as dbert

tokenizer, model = dbert.tebert(model_path)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [6]:
import json

course = json.load(open('/content/drive/MyDrive/smc/data/curricunet/3064?reportId=5.json') )
course

{'course': 'COMPUTER SCIENCE 34A, Game Programming 1',
 'course_objectives': ['Demonstrate an understanding and applications of the asset workflow model',
  'Create a game play',
  'Use 2D and graphics in game development',
  'Use 3D physics in game development',
  'Apply event driven programming to develop games',
  'Apply audio, music and animation in navigation',
  'Adapt games for different platforms such as Mobile devices'],
 'file': 'data/curricunet/3064?reportId=5',
 'student_learning_outcomes': ['Using the principles of Programming, graphics in 2D and 3D and animation, students will develop interactive video games.',
  'Using the tools of a game development tool, students will debug code for logical, syntax and runtime errors.']}

In [7]:
course_text = pd.Series(course['course_objectives'] + course['student_learning_outcomes'])
course_coords = dbert.get_embeddings(course_text, tokenizer, model)


Using cuda:0
0...


## Building a libannoy index of ONET tasks

In [8]:
con = util.onet_db()

In [9]:
tasks = pd.read_sql("select onetsoc_code, task from task_statements where task_type = 'Core'",   con)


In [10]:
import eonacs.common.score as score

In [11]:
task_embeddings =  dbert.get_embeddings(tasks["task"], tokenizer, model)

Using cuda:0
0...
1024...
2048...
3072...
4096...
5120...
6144...
7168...
8192...
9216...
10240...
11264...
12288...


In [12]:
import annoy

In [13]:
task_index = annoy.AnnoyIndex(task_embeddings.shape[1], 'euclidean')

In [15]:
for i in range(task_embeddings.shape[0]):
  task_index.add_item(i, task_embeddings[i,:])

In [16]:
task_index.build(30) #30 trees

True

In [24]:
for i in range(course_coords.shape[0]):
  print(course_text[i])
  i = task_index.get_nns_by_vector(course_coords[i,:], 10)
  print("nearest occupation tasks")
  print(tasks.loc[i,:])
  print()

Demonstrate an understanding and applications of the asset workflow model
nearest occupation tasks
     onetsoc_code                                               task
2685   17-2199.09  Identify new applications for existing nanotec...
2022   15-2051.01  Identify and analyze industry or geographic tr...
3277   19-2041.03  Develop alternative energy investment scenario...
2973   19-1013.00  Develop ways of altering soils to suit differe...
2431   17-2112.03  Incorporate new manufacturing methods or proce...
5336   27-1012.00  Develop product packaging, display, and pricin...
1111   13-1081.01  Design comprehensive supply chains that minimi...
3267   19-2041.03  Identify sustainable alternatives to industria...
1453   13-2072.00  Analyze potential loan markets and develop ref...
2198   17-2051.00  Design energy-efficient or environmentally sou...

Create a game play
nearest occupation tasks
      onetsoc_code                                               task
2661    17-2199.08         